# WHO Models

## Libraries, functions and features selection

In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.tools
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from statsmodels.tools.tools import pinv_extended 
from numpy.linalg import cond
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [26]:
df = pd.read_csv('Life Expectancy Data.csv')
df.head()

,Country,Region,Year,Infant_deaths,Under_five_deaths,Adult_mortality,Alcohol_consumption,Hepatitis_B,Measles,BMI,...,Diphtheria,Incidents_HIV,GDP_per_capita,Population_mln,Thinness_ten_nineteen_years,Thinness_five_nine_years,Schooling,Economy_status_Developed,Economy_status_Developing,Life_expectancy
0,Turkiye,Middle East,2015,11.1,13.0,105.8240,1.32,97,65,27.8,...,97,0.08,11006,78.53,4.9,4.8,7.8,0,1,76.5
1,Spain,European Union,2015,2.7,3.3,57.9025,10.35,97,94,26.0,...,97,0.09,25742,46.44,0.6,0.5,9.7,1,0,82.8
2,India,Asia,2007,51.5,67.9,201.0765,1.57,60,35,21.2,...,64,0.13,1076,1183.21,27.1,28.0,5.0,0,1,65.4
3,Guyana,South America,2006,32.8,40.5,222.1965,5.68,93,74,25.3,...,93,0.79,4146,0.75,5.7,5.5,7.9,0,1,67.0
4,Israel,Middle East,2012,3.4,4.3,57.9510,2.89,97,89,27.0,...,94,0.08,33995,7.91,1.2,1.1,12.8,1,0,81.7


In [133]:
# Splits features and target

def create_ft(df):
    features = df.drop(columns=['Life_expectancy'])
    target = df['Life_expectancy']
    return features, target
    

In [6]:
def create_train_test(features, target):
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

In [134]:
def feature_eng(df):
    df.drop(columns=['Economy_status_Developing', 'Region', 'Country'], inplace=True)
    df['GDP_per_capita_log'] = np.log(df['GDP_per_capita'])
    df.drop(columns=['GDP_per_capita'], inplace=True)
    return df

In [8]:
def stepwise_selection(X, y, threshold_in = 0.01, threshold_out = 0.05, verbose = True):
    # The function is checking for p-values (whether features are statistically significant) - lower is better
    included = [] # this is going to be the list of features we keep
    while True:
        changed = False
        # forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index = excluded, dtype = 'float64')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        # we add the feature with the lowest (best) p-value under the threshold to our 'included' list
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval)) # specifying the verbose text


        # backward step: removing features if new features added to the list make them statistically insignificant
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        # if the p-value exceeds the upper threshold, the feature will be dropped from the 'included' list
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [70]:

def calculate_vif(X):
    X = sm.add_constant(X)    # Adds a constant to DataFrame X so that the function "variance_inflation_factor" can perform its tests
    vif_data = pd.DataFrame()    # Creates DataFrame that will be used to visualize vif data
    vif_data['Variable'] = X.columns
    vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

In [204]:
# Call the function to split features and target from DataFrame
features, target = create_ft(df)
# Train-test splitting
X_train, X_test, y_train, y_test = create_train_test(features, target)
# Feature engineer train and test features indepedently
X_train = feature_eng(X_train)
X_test = feature_eng(X_test)
# Selects features based on p-values contributing to the model
selected_features = stepwise_selection(X_train, y_train, threshold_in=0.01, threshold_out=0.05)
# Trims features to those selected
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]


Add  Under_five_deaths              with p-value 0.0
Add  Adult_mortality                with p-value 0.0
Add  Economy_status_Developed       with p-value 1.79712e-150
Add  GDP_per_capita_log             with p-value 2.42674e-52
Add  Infant_deaths                  with p-value 2.61209e-14
Add  BMI                            with p-value 1.03723e-10
Add  Schooling                      with p-value 8.55602e-11
Add  Thinness_ten_nineteen_years    with p-value 1.50631e-06
Add  Year                           with p-value 0.000120534
Add  Alcohol_consumption            with p-value 0.000514411
Add  Incidents_HIV                  with p-value 0.00695664
Add  Hepatitis_B                    with p-value 0.00628234
Add  Polio                          with p-value 0.00171435


In [205]:
# Runs vif tests to identify multicollinearity
vif_data = calculate_vif(X_train_selected)
vif_data

,Variable,VIF
0,const,203155.946488
1,Under_five_deaths,45.179675
2,Adult_mortality,8.334180
3,Economy_status_Developed,2.753713
4,GDP_per_capita_log,5.126240
5,Infant_deaths,47.007928
6,BMI,2.776837
7,Schooling,4.395716
8,Thinness_ten_nineteen_years,1.943154
9,Year,1.088183


In [206]:
# Drops collinear variable
selected_features.remove('Infant_deaths')
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

In [207]:
X_train_scale = X_train_selected.copy()
X_test_scale = X_test_selected.copy()

In [208]:
# Train MinMax Scaler on training data only (!)
minmax = MinMaxScaler()
minmax.fit(X_train_scale)

MinMaxScaler()

In [209]:
# Perform minmaxing transformation
X_train_scale[selected_features] = minmax.transform(X_train_scale)

In [210]:
# Repeat above for testing data
X_test_scale[selected_features] = minmax.transform(X_test_scale)

## Full Model

In [212]:
def train_model(X_train_scale, y_train):

    X_train_scale = sm.add_constant(X_train_scale) # Adds constant to fully transformed data
    model = sm.OLS(y_train, X_train_scale).fit() # Train linear regression model

    # Metrics
    train_rmse = statsmodels.tools.eval_measures.rmse(y_train, model.predict(X_train_scale))
    train_mae = statsmodels.tools.eval_measures.meanabs(y_train, model.predict(X_train_scale))
    vare = statsmodels.tools.eval_measures.vare(y_train, model.predict(X_train_scale))
    print(f'Train Root Mean Squared Error: {train_rmse:.5f}')
    print(f'Train Mean Absolute Error: {train_mae:.5f}')
    print(f'Train Variance Explained: {vare:.5f}')


    return model

def evaluate_model(model, X_test_scale, y_test):
    
    X_test_scale = sm.add_constant(X_test_scale)
    predictions = model.predict(X_test_scale)
    results = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})

    rmse = statsmodels.tools.eval_measures.rmse(y_test, predictions)
    mae = statsmodels.tools.eval_measures.meanabs(y_test, predictions)
    vare = statsmodels.tools.eval_measures.vare(y_test, predictions)

    print(f'Root Mean Squared Error: {rmse:.5f}') 
    print(f'Mean Absolute Error: {mae:.5f}')
    print(f'Variance Explained: {vare:.5f}')


model = train_model(X_train_scale, y_train)
evaluate_model(model, X_test_scale, y_test)

model.summary()

Train Root Mean Squared Error: 1.34837
Train Mean Absolute Error: 1.06746
Train Variance Explained: 1.81810
Root Mean Squared Error: 1.35186
Mean Absolute Error: 1.08133
Variance Explained: 1.82469


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.980
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     9184.
Date:                Thu, 10 Jul 2025   Prob (F-statistic):               0.00
Time:                        18:47:37   Log-Likelihood:                -3935.6
No. Observations:                2291   AIC:                             7897.
Df Residuals:                    2278   BIC:                             7972.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          76.3813      0.379    201.468      0.000      75.638      77.125
Under_five_deaths             -16.8442      0.401    -41.972      0.000     -17.631     -16.057
Adult_mortality               -30.8633      0.461    -67.011      0.000     -31.766     -29.960
Economy_status_Developed        0.8520      0.115      7.400      0.000       0.626       1.078
GDP_per_capita_log              3.9510      0.283     13.942      0.000       3.395       4.507
BMI                            -2.4692      0.263     -9.399      0.000      -2.984      -1.954
Schooling                       0.9995      0.239      4.176      0.000       0.530       1.469
Thinness_ten_nineteen_years    -1.3312      0.246     -5.402      0.000      -1.814      -0.848
Year                            0.4487      0.095      4.728      0.000       0.263       0.635
Alcohol_consumption             0.8901      0.197      4.525      0.000       0.504       1.276
Incidents_HIV                   1.2507      0.453      2.759      0.006       0.362       2.140
Hepatitis_B                    -0.8044      0.225     -3.574      0.000      -1.246      -0.363
Polio                           1.1625      0.330      3.528      0.000       0.516       1.809
==============================================================================
Omnibus:                       25.804   Durbin-Watson:                   1.972
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               35.160
Skew:                           0.140   Prob(JB):                     2.32e-08
Kurtosis:                       3.538   Cond. No.                         44.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [213]:
def ridge_model(features, target):
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size=0.2, random_state=42
    )

    X_train = feature_eng(X_train)
    X_test = feature_eng(X_test)

    x_scaler = RobustScaler()
    y_scaler = RobustScaler()

    X_train_scaled = x_scaler.fit_transform(X_train)
    X_test_scaled = x_scaler.transform(X_test)

    y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
    y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1)).flatten()

    model = Ridge(alpha=0.1)
    model.fit(X_train_scaled, y_train_scaled)

    predictions_scaled = model.predict(X_test_scaled)
    predictions = y_scaler.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()

    rmse = statsmodels.tools.eval_measures.rmse(y_test, predictions)
    print(f'Ridge Root Mean Squared Error: {rmse:.5f}')

    r_squared = model.score(X_test_scaled, y_test_scaled)
    print(f'Ridge R^2 (scaled): {r_squared:.5f}')

    condition_number = np.linalg.cond(X_train_scaled)
    print(f'Ridge Condition Number: {condition_number:.5f}')

    return model, x_scaler, y_scaler, X_train_scaled, X_test_scaled, y_test

def evaluate_ridge_model(model, X_test_scaled, y_test_scaled, y_scaler):
    predictions_scaled = model.predict(X_test_scaled)

    predictions = y_scaler.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()
    actual = y_scaler.inverse_transform(y_test_scaled.values.reshape(-1, 1)).flatten()

    rmse = statsmodels.tools.eval_measures.rmse(actual, predictions)
    mae = statsmodels.tools.eval_measures.meanabs(actual, predictions)
    vare = statsmodels.tools.eval_measures.vare(actual, predictions)

    print(f'Ridge Test Root Mean Squared Error: {rmse:.5f}') 
    print(f'Ridge Test Mean Absolute Error: {mae:.5f}')
    print(f'Ridge Test Variance Explained: {vare:.5f}')

    return pd.DataFrame({'Actual': actual, 'Predicted': predictions})

model2, x_scaler, y_scaler, X_train_scaled, X_test_scaled, y_test_scaled = ridge_model(features, target)

Ridge Root Mean Squared Error: 1.34905
Ridge R^2 (scaled): 0.97807
Ridge Condition Number: 81.81233


## Slim Model (non-medical records)

In [214]:
# Drops medical variables
medical_features = ['BMI','Incidents_HIV','Polio','Hepatitis_B','Thinness_ten_nineteen_years']
for item in medical_features:
    selected_features.remove(item)
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

In [215]:
X_train_scale2 = X_train_selected.copy()
X_test_scale2 = X_test_selected.copy()

In [216]:
# Train MinMax Scaler on a second selection of features
minmax2 = MinMaxScaler()
minmax2.fit(X_train_scale2)

MinMaxScaler()

In [217]:
# Perform minmaxing transformation
X_train_scale2[selected_features] = minmax2.transform(X_train_scale2)

In [218]:
# Perform minmaxing transformation
X_test_scale2[selected_features] = minmax2.transform(X_test_scale2)

In [219]:
model = train_model(X_train_scale2, y_train)
evaluate_model(model, X_test_scale2, y_test)

model.summary()

Train Root Mean Squared Error: 1.38200
Train Mean Absolute Error: 1.08992
Train Variance Explained: 1.90994
Root Mean Squared Error: 1.39692
Mean Absolute Error: 1.10519
Variance Explained: 1.95074


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                 1.500e+04
Date:                Thu, 10 Jul 2025   Prob (F-statistic):               0.00
Time:                        18:47:56   Log-Likelihood:                -3992.0
No. Observations:                2291   AIC:                             8000.
Df Residuals:                    2283   BIC:                             8046.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       75.6883      0.188    402.628      0.000      75.320      76.057
Under_five_deaths          -16.9706      0.331    -51.233      0.000     -17.620     -16.321
Adult_mortality            -29.9728      0.303    -98.792      0.000     -30.568     -29.378
Economy_status_Developed     1.2061      0.113     10.694      0.000       0.985       1.427
GDP_per_capita_log           3.4130      0.258     13.223      0.000       2.907       3.919
Schooling                    0.6785      0.234      2.903      0.004       0.220       1.137
Year                         0.3419      0.096      3.561      0.000       0.154       0.530
Alcohol_consumption          1.0791      0.197      5.484      0.000       0.693       1.465
==============================================================================
Omnibus:                       25.210   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.328
Skew:                           0.057   Prob(JB):                     1.75e-09
Kurtosis:                       3.640   Cond. No.                         21.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""